In [ ]:
%%capture
import pandas as pd
from django_pandas.io import read_frame
from pathlib import Path
from dj_notebook import activate

plus = activate(dotenv_file="/Users/erikvw/source/edc_source/meta-edc/.env")
report_folder = Path("/Users/erikvw/Documents/ucl/protocols/meta3/reports/")
# output is suppressed -- normally would spew out all the edc loading messages


In [ ]:
from edc_registration.models import RegisteredSubject
from edc_appointment.analytics import get_appointment_df
from meta_prn.models import OnSchedule, OffSchedule, OffSchedulePregnancy, OffSchedulePostnatal, OnScheduleDmReferral, \
    EndOfStudy

df_onschedule = read_frame(OnSchedule.objects.all(), verbose=True)
df_offschedule = read_frame(OffSchedule.objects.all(), verbose=True)
df_onschedule_preg = read_frame(OffSchedulePregnancy.objects.all(), verbose=True)
df_onschedule_postnatal = read_frame(OffSchedulePostnatal.objects.all(), verbose=True)
df_onschedule_dm = read_frame(OnScheduleDmReferral.objects.all(), verbose=True)
df_eos = read_frame(EndOfStudy.objects.all(), verbose=True)
df_appt = get_appointment_df()
df_rs = read_frame(RegisteredSubject.objects.values("subject_identifier", "registration_datetime").all(), verbose=True)


In [ ]:
df_appt[df_appt.visit_code > 1360][["visit_code", "schedule_name"]].schedule_name.value_counts()

In [ ]:
df_appt[(df_appt.visit_code > 1360) & (df_appt.schedule_name == "schedule")][["visit_code", "schedule_name"]].visit_code.value_counts()

In [ ]:
df_appt[(df_appt.visit_code > 1360) & (df_appt.schedule_name == "schedule")][["subject_identifier", "appt_datetime", "visit_code", "appt_status"]].sort_values("subject_identifier")

In [ ]:
df_appt = get_appointment_df()

df_appt = df_appt[(df_appt.visit_code >= 1360) & (df_appt.schedule_name == "schedule")][["subject_identifier", "appt_datetime", "visit_code", "appt_status"]]

In [ ]:
df_magreth = pd.read_csv(Path("/Users/erikvw/Documents/ucl/protocols/meta3/reports/") / "consented_v1_ext_magreth.csv")


In [ ]:
df_magreth

In [ ]:
df_main = pd.merge(df_appt[["subject_identifier", "appt_datetime", "visit_code", "appt_status"]], df_magreth, on="subject_identifier", how="right").groupby(["subject_identifier", "agreed"]).agg({"visit_code": "min", "date_reconsented": "max"})

In [ ]:
df_main = df_main.merge(df_appt[["subject_identifier", "visit_code", "appt_status", "appt_datetime"]], on=["subject_identifier", "visit_code"], how="left")
df_main


In [ ]:
df_rs

In [ ]:
from meta_consent.models import SubjectConsentV1Ext
df_v1ext = read_frame(SubjectConsentV1Ext.objects.values("subject_identifier", "report_datetime", "agrees_to_extension").all(), verbose=True)
df_v1ext

In [ ]:
df_main = df_main.merge(df_rs, on="subject_identifier", how="left")

In [ ]:
df_main = df_main.merge(df_v1ext, on="subject_identifier", how="outer")
df_main.rename(columns={"report_datetime": "v1_ext_datetime", "agrees_to_extension": "agreed", "visit_code": "last_visit_code"}, inplace=True)


In [ ]:
df_main = df_main.merge(df_eos[["subject_identifier", "offstudy_datetime"]], on="subject_identifier", how="left")


In [ ]:
df_main[["subject_identifier","registration_datetime", "last_visit_code", "appt_status", "appt_datetime", "date_reconsented", "v1_ext_datetime", "agreed"]]